In [1]:
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
import scipy.io
import time

class PhysicsInformedNN(nn.Module):
    def __init__(self, x0, u0, x1, u1, layers, dt, lb, ub, q):
        super(PhysicsInformedNN, self).__init__()
        self.lb = torch.tensor(lb, dtype=torch.float32)
        self.ub = torch.tensor(ub, dtype=torch.float32)
        
        self.x0 = torch.tensor(x0, dtype=torch.float32, requires_grad=True)
        self.x1 = torch.tensor(x1, dtype=torch.float32, requires_grad=True)
        
        self.u0 = torch.tensor(u0, dtype=torch.float32)
        self.u1 = torch.tensor(u1, dtype=torch.float32)
        
        self.dt = torch.tensor(dt, dtype=torch.float32)
        self.q = max(q, 1)

        # Initialize NN
        self.model = self.build_nn(layers)
        
        # Parameters to be learned
        self.lambda_1 = nn.Parameter(torch.tensor(0.75))
        self.lambda_2 = nn.Parameter(torch.tensor(-6.0))

        # IRK weights
        tmp = np.float32(np.loadtxt('IRK_weights/Butcher_IRK%d.txt' % (q), ndmin=2))
        weights = np.reshape(tmp[0:q**2+q], (q+1, q))
        self.IRK_alpha = torch.tensor(weights[0:-1, :], dtype=torch.float32)
        self.IRK_beta = torch.tensor(weights[-1:, :], dtype=torch.float32)
        self.IRK_times = torch.tensor(tmp[q**2+q:], dtype=torch.float32)

    def build_nn(self, layers):
        modules = []
        for i in range(len(layers) - 1):
            modules.append(nn.Linear(layers[i], layers[i + 1]))
            if i < len(layers) - 2:
                modules.append(nn.Tanh())
        return nn.Sequential(*modules)

    def forward(self, x):
        x = 2.0 * (x - self.lb) / (self.ub - self.lb) - 1.0
        return self.model(x)

    def fwd_gradients(self, U, x):
        return torch.autograd.grad(U, x, grad_outputs=torch.ones_like(U), create_graph=True)[0]

    def net_U0(self, x):
        U = self.forward(x)
        U_x = self.fwd_gradients(U, x)
        U_xx = self.fwd_gradients(U_x, x)
        U_xxx = self.fwd_gradients(U_xx, x)

        F = -self.lambda_1 * U * U_x - torch.exp(self.lambda_2) * U_xxx
        U0 = U - self.dt * torch.matmul(F, self.IRK_alpha.T)
        return U0

    def net_U1(self, x):
        U = self.forward(x)
        U_x = self.fwd_gradients(U, x)
        U_xx = self.fwd_gradients(U_x, x)
        U_xxx = self.fwd_gradients(U_xx, x)

        F = -self.lambda_1 * U * U_x - torch.exp(self.lambda_2) * U_xxx
        U1 = U + self.dt * torch.matmul(F, (self.IRK_beta - self.IRK_alpha).T)
        return U1

    def loss_fn(self):
        U0_pred = self.net_U0(self.x0)
        U1_pred = self.net_U1(self.x1)

        loss = torch.mean((self.u0 - U0_pred) ** 2) + torch.mean((self.u1 - U1_pred) ** 2)
        return loss

    def train(self, nIter):
        optimizer = torch.optim.Adam(self.parameters(), lr=1e-3)
        for it in range(nIter):
            optimizer.zero_grad()
            loss = self.loss_fn()
            loss.backward()
            optimizer.step()

            if it % 10 == 0:
                print(f"Iteration {it}, Loss: {loss.item():.3e}, l1: {self.lambda_1.item():.3f}, l2: {torch.exp(self.lambda_2).item():.5f}")

    def predict(self, x_star):
        x_star = torch.tensor(x_star, dtype=torch.float32, requires_grad=True)
        U0_pred = self.net_U0(x_star)
        U1_pred = self.net_U1(x_star)
        return U0_pred.detach().numpy(), U1_pred.detach().numpy()


if __name__ == "__main__":
    q = 50
    skip = 120

    N0 = 199
    N1 = 201
    layers = [1, 50, 50, 50, 50, q]

    data = scipy.io.loadmat('KdV.mat')

    t_star = data['tt'].flatten()[:, None]
    x_star = data['x'].flatten()[:, None]
    Exact = np.real(data['uu'])

    idx_t = 40

    noise = 0.01
    idx_x = np.random.choice(Exact.shape[0], N0, replace=False)
    x0 = x_star[idx_x, :]
    u0 = Exact[idx_x, idx_t][:, None]
    u0 += noise * np.std(u0) * np.random.randn(*u0.shape)

    idx_x = np.random.choice(Exact.shape[0], N1, replace=False)
    x1 = x_star[idx_x, :]
    u1 = Exact[idx_x, idx_t + skip][:, None]
    u1 += noise * np.std(u1) * np.random.randn(*u1.shape)

    dt = t_star[idx_t + skip] - t_star[idx_t]

    lb = x_star.min(0)
    ub = x_star.max(0)

    model = PhysicsInformedNN(x0, u0, x1, u1, layers, dt, lb, ub, q)
    model.train(nIter=500)

    U0_pred, U1_pred = model.predict(x_star)

    lambda_1_value = model.lambda_1.item()
    lambda_2_value = torch.exp(model.lambda_2).item()

    print(f"Identified lambda_1: {lambda_1_value}")
    print(f"Identified lambda_2: {lambda_2_value}")


Iteration 0, Loss: 9.899e-01, l1: 0.749, l2: 0.00248
Iteration 10, Loss: 9.577e-01, l1: 0.739, l2: 0.00251
Iteration 20, Loss: 9.317e-01, l1: 0.730, l2: 0.00253
Iteration 30, Loss: 8.464e-01, l1: 0.737, l2: 0.00256
Iteration 40, Loss: 8.149e-01, l1: 0.728, l2: 0.00260
Iteration 50, Loss: 7.734e-01, l1: 0.720, l2: 0.00263
Iteration 60, Loss: 7.285e-01, l1: 0.714, l2: 0.00266
Iteration 70, Loss: 6.959e-01, l1: 0.715, l2: 0.00269
Iteration 80, Loss: 6.697e-01, l1: 0.721, l2: 0.00271
Iteration 90, Loss: 6.413e-01, l1: 0.732, l2: 0.00274
Iteration 100, Loss: 6.046e-01, l1: 0.744, l2: 0.00277
Iteration 110, Loss: 5.365e-01, l1: 0.757, l2: 0.00281
Iteration 120, Loss: 4.878e-01, l1: 0.772, l2: 0.00279
Iteration 130, Loss: 4.757e-01, l1: 0.782, l2: 0.00275
Iteration 140, Loss: 4.676e-01, l1: 0.792, l2: 0.00271
Iteration 150, Loss: 4.558e-01, l1: 0.802, l2: 0.00268
Iteration 160, Loss: 4.469e-01, l1: 0.812, l2: 0.00265
Iteration 170, Loss: 4.401e-01, l1: 0.822, l2: 0.00262
Iteration 180, Loss: 

In [3]:
error_lambda_1 = np.abs(lambda_1_value - 1.0)/1.0 *100
error_lambda_2 = np.abs(lambda_2_value - 0.0025)/0.0025 * 100
    
print('Error lambda_1: %f%%' % (error_lambda_1))
print('Error lambda_2: %f%%' % (error_lambda_2))

Error lambda_1: 0.566000%
Error lambda_2: 3.992414%
